In [6]:
import os
import random
import math

import numpy as np
import haven.db as db 
import plotly.express as px

from mirrorverse.utils import read_data_w_cache

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

In [ ]:
TEST = True

table = 'movement_model_raw_features_m2_a2' if not TEST else 'movement_model_raw_features_m2_a2_test'
data = read_data_w_cache(
    f'select * from {table}'
)
data = data[data['_selected'].isin([True, False])]
data.head()

In [ ]:
df = data.merge(
    data[['_individual']].drop_duplicates().reset_index(drop=True).reset_index(),
    on='_individual'
)
del df['_individual']
df = df.rename({'index': '_individual'}, axis=1)
df.head()

In [ ]:
df.groupby('_train').size()

In [ ]:
df['normed_distance'] = df['distance'] / 100
px.histogram(df['normed_distance'])

In [ ]:
df['normed_log_mlt'] = np.log(df['mixed_layer_thickness'] + 0.001)
print(df['normed_log_mlt'].mean())
df['normed_log_mlt'] = df['normed_log_mlt'] - 3.068049438368808
px.histogram(df['normed_log_mlt'])

In [ ]:
df['normed_log_npp'] = np.log(df['net_primary_production'] + 0.001)
print(df['normed_log_npp'].mean())
df['normed_log_npp'] = df['normed_log_npp'] - 1.9856236
px.histogram(df['normed_log_npp'])

In [20]:
table = 'movement_model_features_m2_a2' if not TEST else 'movement_model_features_m2_a2_test'

db.write_data(
    df, table, ['tag_key']
)